<a href="https://colab.research.google.com/github/aaronjyang/transformers-testing/blob/main/LLaVa-loftus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model and necessary imports

In [1]:
# Pip installs
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/tran

In [2]:
# Imports
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
from PIL import Image
import numpy as np
import av
import numpy as np
from huggingface_hub import hf_hub_download
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
# Load Model

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Load videos


In [4]:
def read_video_pyav(path, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        path (str): Path to the video file.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    container = av.open(path)
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [5]:
videoIDs = ["1080", "720", "480", "360", "240", "144"]
videos = {}
indices = np.arange(0, 51)
for videoID in videoIDs:
  video = read_video_pyav("/content/quality_degradation/" + videoID + "p10fps.mp4", indices)
  videos[videoID] = video

# Test videos (optional)


In [6]:
# np array with shape (frames, height, width, channels)
video = videos["720"]

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=100)
HTML(anim.to_html5_video())

# Misc. Functions

In [28]:
def generate_response(conversation, video, max_new_tokens = 100, do_sample= True, top_p = 0.9):
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor([prompt], videos=video, padding=True, return_tensors="pt").to(model.device)
  generate_kwargs = {"max_new_tokens": max_new_tokens, "do_sample": do_sample, "top_p": top_p}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)
  return generated_text[0][len(prompt) - 7 + 1:]

In [8]:
def ask_question(conversation, video, question):
  next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": question},
          ],
  }
  conversation.append(next_question)
  response = generate_response(conversation, video)
  assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": response},
      ],
  }
  conversation.append(assistant_response)
  return conversation, response

In [9]:
import random
def ask_math_questions(conversation, numQuestions, video):
  for i in range(numQuestions):
    num1 = random.randint(1, 10)
    num2 = random.randint(1, 10)
    next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": "What is " + str(num1) + " + " + str(num2) +
               "? Please answer in the form \"" +  str(num1) + " + " + str(num2) + " = sum\" with no more and no less words."},
          ],
    }
    conversation.append(next_question)

    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    print(len(processor.tokenizer.tokenize(prompt)))
    inputs = processor([prompt], padding=True, return_tensors="pt").to(model.device)
    generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.90}
    output = model.generate(**inputs, **generate_kwargs)
    generated_text = processor.batch_decode(output, skip_special_tokens=True)
    response =  generated_text[0][len(prompt) + 1:]
    assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": response},
      ],
    }
    conversation.append(assistant_response)
  return conversation

# Experiment

In [25]:
def run_experiment(videoID, verb, numTrials = 1, numMathQuestions = 0):
  video = videos[videoID]
  outputs = {}
  questions = [ "Give an account of the accident you have just seen in the provided video.",
              "Does the video happen at day or at night?",
              "What is the weather in the video?",
              "What color were the vehicles involved in the accident?",
              "Were any pedestrians involved in the accident?",
              "From the options, what is your best estimate for how fast were the cars going when they "+verb+" each other \n" +
                "A) 0 - 10 kilometers per hour\n" +
                "B) 10 - 20 kilometers per hour\n" +
                "C) 20 - 30 kilometers per hour\n" +
                "D) 30 - 40 kilometers per hour\n"]
  outputs["videoID"] = []
  outputs["verb"] = []
  outputs["trial #"] = []
  outputs["padding"] = []
  for i in range(numTrials):
    outputs["videoID"].append(videoID);
    outputs["verb"].append(verb)
    outputs["trial #"].append(i)
    outputs["padding"].append(numMathQuestions)
    conversation = []
    conversation = ask_math_questions(conversation, numMathQuestions, video)
    conversation.append(
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Does a car accident occur in the provided video?"},
                {"type": "video"},
                ],
        }
    )
    generated_response = generate_response(conversation, video)
    if "Does a car accident occur in the provided video?" in outputs:
      outputs["Does a car accident occur in the provided video?"].append(generated_response)
    else:
      outputs["Does a car accident occur in the provided video?"] = [generated_response]
    assistant_response = {
        "role": "assistant",
        "content": [
            {"type": "text", "text": generated_response},
        ],
    }
    conversation.append(assistant_response)
    for question in questions:
      conversation, response = ask_question(conversation, video, question);
      if question in outputs:
        outputs[question].append(response)
      else:
        outputs[question] = [response]
  return outputs




In [21]:
o = run_experiment("720", "collided with", 1, 0)

In [29]:
verbs = ["smashed into", "collided with", "bumped into", "contacted", "hit"]
numMath = [0, 15, 30]
outputs = {}
firstTime = True;
for verb in verbs:
  for num in numMath:
    o = run_experiment("720", verb, 1, num)
    if firstTime:
      outputs = o
      firstTime = False
    else:
      for key in o:
        outputs[key].append(o[key])

OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 4593 has 14.73 GiB memory in use. Of the allocated memory 11.23 GiB is allocated by PyTorch, and 3.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
o

{'videoID': ['720'],
 'verd': ['collided with'],
 'trial #': [0],
 'Does a car accident occur in the provided video?': ['Yes, a car accident occurs in the video. The car crashes into a truck, and a person is visible in the vehicle. The scene depicted in the video is a collision involving a red car that crashes into a white truck.'],
 'Give an account of the accident you have just seen in the provided video.': ["The video captures a car accident in which a red car crashes into a white truck, causing it to roll and spin. As a result, the car becomes stuck to the truck, and the driver's condition is unknown after the collision. The truck is also damaged and the accident scene is chaotic, with people in the vehicles and visible debris."],
 'Does the video happen at day or at night?': ['The video shows the car accident at night, but the specific time or hour of the day is not provided. The lighting conditions make it difficult to discern the exact time, but it is not possible to determine t

In [23]:
import pandas as pd
pd.DataFrame(o)

,videoID,verd,trial #,Does a car accident occur in the provided video?,Give an account of the accident you have just seen in the provided video.,Does the video happen at day or at night?,What is the weather in the video?,What color were the vehicles involved in the accident?,Were any pedestrians involved in the accident?,"From the options, about how fast were the cars going when they collided with each other \nA) 0 - 10 kilometers per hour\nB) 10 - 20 kilometers per hour\nC) 20 - 30 kilometers per hour\nD) 30 - 40 kilometers per hour\n"
0,720,collided with,0,"Yes, a car accident occurs in the video. The c...",The video captures a car accident in which a r...,"The video shows the car accident at night, but...","The weather in the video appears to be sunny, ...",The vehicles involved in the accident are red ...,"No, the video only shows the vehicles involved...",Given the context of the accident occurring at...


In [ ]:
for question in questions:
  next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": question},
          ],
  }
  conversation.append(next_question)
  response = generate_response(conversation, videos["720"])
  outputs[question] = response

  assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": response},
      ],
  }
  conversation.append(assistant_response)


In [ ]:
conversation

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Does a car accident occur in the provided video?'},
   {'type': 'video'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'Yes, a car accident does occur in the video, as a car is seen colliding with another vehicle while passing it.'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'Give an account of the accident you have just seen in the provided video.'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': "In the video, a car is seen accelerating and passing another vehicle. However, it doesn't pass by safely and collides with the second vehicle, resulting in an accident. The video clearly shows the collision and the damages sustained by both the vehicles. It's important to note that accidents can occur in any situation, and the importance of obeying traffic rules and being vigilant is always emphasized."}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text':

In [ ]:
outputs

{'Does a car accident occur in the provided video?': 'Yes, a car accident does occur in the video, as a car is seen colliding with another vehicle while passing it.',
 'Give an account of the accident you have just seen in the provided video.': "In the video, a car is seen accelerating and passing another vehicle. However, it doesn't pass by safely and collides with the second vehicle, resulting in an accident. The video clearly shows the collision and the damages sustained by both the vehicles. It's important to note that accidents can occur in any situation, and the importance of obeying traffic rules and being vigilant is always emphasized.",
 'Does the video happen at day or at night?': 'The video appears to take place during the day, as the lighting is bright and clear, making it possible to see the details of the accident and the surrounding environment.',
 'What is the weather in the video?': 'The weather in the video appears to be clear and sunny, indicating that the video migh

In [ ]:
conversation = [{
      "role": "user",
          "content": [
              {"type": "text", "text": "What is 2 + 1?"},
          ],
  }]

In [ ]:
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor([prompt], padding=True, return_tensors="pt").to(model.device)
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.90}
output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=True)

In [ ]:
generated_text

['USER: What is 2 + 1? ASSISTANT: The sum of 2 and 1 is 3. округу']

In [ ]:
conversation = ask_math_questions(conversation, 20)

TypeError: ask_math_questions() missing 1 required positional argument: 'video'

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()
del model